In [381]:
import requests
import pandas as pd
import urllib
import os
from bs4 import BeautifulSoup
import xlrd
import numpy as np


# Download URL found using Chrome Console and recording network activity
DATA_URL = "https://sos.iowa.gov/elections/results/xls/2018/general/"
RAW_DEST = "raw-from-source/SoS_raw/"
PROCESSED_DEST = "raw-from-source/SoS_processed/"
CSV_DEST = "raw-from-source/SoS_converted/"
FINAL_DEST = "raw-from-source/"

In [2]:
# List of all counties in Iowa taken from wikipedia
county_list = open("counties.txt", "r").readlines()
county_list = [c.strip() for c in county_list]
county_list = [c.replace(" ", "%20") + ".xls" for c in county_list]

In [156]:
# Download xls file for each county, save locally
for county in county_list:
    print("Downloading county", county[:-4].replace("%20", " "))
    if os.path.exists(RAW_DEST + county):
        print("Local copy of data already downloaded.")
    else:
        print("Requesting from SoS website.")
        urllib.request.urlretrieve(DATA_URL + county, RAW_DEST + county)
    print()

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.

Local copy of data already downloaded.



#### VEST data used the Gov, AG, SOS, Treasurer, Auditor, and Ag Sec Race.

In [384]:
# TODO, comment this and explain what's going on

name2vest = {
    "Fred Hubbell/Rita R. Hart" : "G18GOVDHUB",
    "Kim Reynolds/Adam Gregg" : "G18GOVRREY",
    "Jake Porter/Lynne Gentry" : "G18GOVLPOR",
    "Gary Siegwarth/Natalia Blaskovich" : "G18GOVOSIE",
    
    "Tom Miller" : "G18ATGDMIL",
    "Marco Battaglia" : "G18ATGLBAT",
    
    "Paul D. Pate" : "G18SOSRPAT",
    "Deidre DeJear" : "G18SOSDDEJ",
    "Jules Ofenbakh" : "G18SOSLOFE",
    
    "Mike Naig" : "G18AGRRNAI",
    "Tim Gannon" : "G18AGRDGAN",
    "Rick Stewart" : "G18AGRLSTE",
    
    "Jeremy N. Davis" : "G18TRERDAV",
    "Michael L. Fitzgerald" : "G18TREDFIT",
    "Timothy Hird" : "G18TRELHIR",
    
    "Mary Mosiman" : "G18AUDRMOS",
    "Rob Sand" : "G18AUDDSAN",
    "Fred Perryman" : "G18AUDLPER", 
}

##display(df.head(50))
positions_of_interest = ["Attorney General ( 1)", "Auditor of State ( 1)", "Secretary of State ( 1)", 
                             "Governor/Lieutenant Governor ( 1)", "Secretary of Agriculture ( 1)", 
                                 "Treasurer of State ( 1)"]


pos2idx = {pos : idx for idx, pos in enumerate(positions_of_interest)}
# Okay, now we need to filter this to only capture the elections we care about
problem_counties = []
def process_and_save_df(df, county):

    df.rename(columns={"Unnamed: 0" : "Race"}, inplace=True)
    idxs = np.where(df.Race == "NEW RACE")
    built_df = [0] * len(positions_of_interest)
    start = -1
    
    for i, idx in enumerate(idxs[0]):
        temp = df.iloc[start + 1:idx, :].copy()
        start = idx
        temp.index = [j for j in range(len(temp))]
        position = temp.iloc[0, 0]

        if position not in positions_of_interest:
            continue
                  
        temp.rename(columns={old : old + '_' + str(i) for old in temp}, inplace=True)
        built_df[pos2idx[position]] = temp

    max_dims = (0, 0)
    for d in built_df:
        max_x, max_y = max_dims
        max_dims = (max(max_x, d.shape[0]), max(max_y, d.shape[1]))

    
    combined_df = built_df[0].join([d for d in built_df[1:] if d.shape == max_dims])
    
    # Drop empty rows
    combined_df.dropna(how="all", axis=1, inplace=True)

    prev = float("Nan")
    skip_idxs = []
    for idx, val in enumerate(combined_df.iloc[1][:-1]):
        if idx in skip_idxs:
            continue

        if val == "Write-in":
            try:
                combined_df.iloc[1, idx+2] = prev[:-4] + "OWRI"
                combined_df.iloc[1, idx] = prev[:-4] + "OWRI"
                skip_idxs.append(idx + 2)
                prev = float("Nan")
            except TypeError:
                problem_counties.append(county)
                continue
        
        elif val in name2vest:
            combined_df.iloc[1, idx] = name2vest[val]
            prev = name2vest[val]
            
        else:
            combined_df.iloc[1, idx] = prev
            
            
    # Step 2, get rid of all that aren't total votes
    combined_df.iloc[1, 1] = "Precinct"
    cols_to_drop = []
    for col in combined_df:
        if "Total Votes" not in combined_df[col].values and \
                "Precinct" not in combined_df[col].values:
            cols_to_drop.append(col)
    
    #print("Dropping this many colonies", len(cols_to_drop))
    combined_df.dropna(how="all", axis=1, inplace=True)
    #display(combined_df.iloc[0:5, 0:20])
    # Some final cleaning
    combined_df.drop(columns=cols_to_drop, axis=1, inplace=True)
    combined_df.columns = combined_df.iloc[1]
    #display(combined_df.iloc[0:5, 0:20])
    
    combined_df.drop([0, 1, 2], inplace=True, axis=0)
    combined_df.dropna(how="all", axis=1, inplace=True)
    #display(combined_df.iloc[0:5, 0:20])
    summary_idxs = np.where(combined_df["Precinct"] == "Total:")
    combined_df.index = [i for i in range(len(combined_df))]
    combined_df.drop(summary_idxs[0], inplace=True, axis=0)
    #display(combined_df.iloc[0:5, 0:20])
    
    print("Combined df dimensions", combined_df.shape)
    combined_df["County"] = county
    #print(combined_df.columns)
    
    #display(combined_df.iloc[0:5, 0:30])
    combined_df.to_csv(CSV_DEST + county + ".csv")
    return combined_df

In [353]:

from xlwt import Workbook
import io

count = 0
for county in county_list:
    
    file = io.open(RAW_DEST + county, "r", encoding="utf-8-sig")
    data = file.readlines()
    xldoc = Workbook()
    sheet = xldoc.add_sheet("Sheet1", cell_overwrite_ok=True)
    
    care = False
    in_row = False
    extra_cols = False
    num_cols = 0
    
    row = []
    r_idx = 1
    
    
    first_table = True
    
    for line in data:
        line = line.strip()
        # filter out the beginning metadata
        if '<s:Worksheet s:Name="2">' in line:
            care = True
        if not care:
            continue
            
        if "Table" in line and first_table:
            first_table = False
         
        if "/s:Table" in line and not first_table:
            sheet.write(r_idx, 0, "NEW RACE")
            r_idx += 1
        
        #print("Line:", line)
        if "<s:Row>" in line:
            in_row = True
            
        if "</s:Row>" in line:
            in_row = False
            
            header_row = [elem for elem in row if elem != ""]
            
            if len(header_row) > 1:
                for x_idx, elem in enumerate(row):
                    try: 
                        sheet.write(r_idx, x_idx + 1, elem)
                    except ValueError:
                        continue
            else:
                for x_idx, elem in enumerate(row):
                    sheet.write(r_idx, x_idx, elem)
            
            r_idx += 1
            row = []
            
        if "Cell" in line and "MergeAcross" not in line:
            continue
            
        if "MergeAcross" in line:
            extra_cols = True
            num_cols = line.split('=')[1].split('"')[1]
            num_cols = int(num_cols)
            
        if in_row and "Data" in line:
            val = line.split('>')[1].split('<')[0]
            row.append(val)
            if extra_cols:
                for i in range(num_cols):
                    row.append("")
                extra_cols = False
                num_cols = 0
                

    xldoc.save(COMBINED_DEST + county)
    df = pd.read_excel(COMBINED_DEST + county)
    print("County:", county[:-4].replace("%20", "_"))
    print('-' * 20)
    combo_df = process_and_save_df(df, county[:-4].replace("%20", "_"))
    print()
#     if count > 3:
#         break
#     count += 1
 

County: Adair
--------------------
Combined df dimensions (5, 25)

County: Adams
--------------------
Combined df dimensions (5, 25)

County: Allamakee
--------------------
Combined df dimensions (11, 25)

County: Appanoose
--------------------
Combined df dimensions (12, 25)

County: Audubon
--------------------
Combined df dimensions (2, 25)

County: Benton
--------------------
Combined df dimensions (19, 25)

County: Black_Hawk
--------------------
Combined df dimensions (62, 25)

County: Boone
--------------------
Combined df dimensions (15, 25)

County: Bremer
--------------------
Combined df dimensions (13, 25)

County: Buchanan
--------------------
Combined df dimensions (15, 25)

County: Buena_Vista
--------------------
Combined df dimensions (10, 25)

County: Butler
--------------------
Combined df dimensions (8, 25)

County: Calhoun
--------------------
Combined df dimensions (10, 25)

County: Carroll
--------------------
Combined df dimensions (13, 25)

County: Cass
--------

In [383]:
dfs = [pd.read_csv(CSV_DEST + file) for file in os.listdir(CSV_DEST)]
expecting = 0
# for df in dfs:
#     print(df.shape, len(df.columns.unique()))
#     expecting += df.shape[0]
    
complete_df = pd.concat(dfs, axis=0)
cols_to_remove = [col for col in complete_df if "Unnamed" in col]

complete_df.drop(columns=cols_to_remove, axis=0, inplace=True)

print(complete_df.shape)
display(complete_df.head(5))
complete_df.to_csv(FINAL_DEST + "SoS_processed_election_final.csv")
#complete_df = 5

(1677, 26)


,Precinct,G18ATGDMIL,G18ATGLBAT,G18ATGOWRI,G18AUDRMOS,G18AUDDSAN,G18AUDLPER,G18AUDOWRI,G18SOSRPAT,G18SOSDDEJ,...,G18GOVOWRI,G18AGRRNAI,G18AGRDGAN,G18AGRLSTE,G18AGROWRI,G18TRERDAV,G18TREDFIT,G18TRELHIR,G18TREOWRI,County
0,Geneva / Ingham / East Mott,205,59,1,193,107,2,0,213,86,...,0.0,209,92,5,0,176,127,1,1,Franklin
1,Grant / Osceola,158,50,3,168,80,2,0,185,62,...,0.0,189,61,4,0,161,89,3,0,Franklin
2,Hamilton / Lee / Reeve,175,71,1,189,92,7,0,214,74,...,0.0,210,73,6,0,182,103,5,0,Franklin
3,Hampton 1,201,49,0,124,136,11,0,146,117,...,0.0,138,120,15,0,116,145,9,0,Franklin
4,Hampton 2,257,71,3,203,163,11,0,227,139,...,0.0,232,132,13,0,187,177,6,0,Franklin
